In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['basePart'].tolist()
list_url = ['https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=' + _ for _ in list_url]

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests
import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = json.loads(requests.get(url,
                                               headers=UA.get_User_Agent_Requests(),
                                               proxies=Proxy.get_Proxy_Requests()).text)
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for _ in resp['pp']['buyersGuides']:
                    list_vehicle.append(_['buyersGuideDesc'])

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Part_Number': resp['pd']['basePart'],
                                         'Part_Type': resp['pd']['partDesc_en'],
                                         'Part_Type_Long': resp['pd']['partLongDesc_en'],
                                         'Brand': resp['pd']['brand'],
                                         'Pop': resp['pd']['pop'],
                                         'Per_Car': resp['pd']['qtyEach'],
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': 'https://ecatalog.smpcorp.com/V2/STD/#/partdetail/part/' + resp['pd']['basePart'],
                                         'Src': ''}])
                
                for name_value in resp['pp']['partSpecs']:
                    df_temp[name_value['attributeName_en'].lower()] = name_value['siteValue_en']
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：498

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS101  <->  [ok] - 剩余数量：493
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APK101  <->  [ok] - 剩余数量：492
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APK102  <->  [ok] - 剩余数量：491
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APK100  <->  [ok] - 剩余数量：490
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS104  <->  [ok] - 剩余数量：489
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS103  <->  [ok] - 剩余数量：488
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS107  <->  [ok] - 剩余数量：487
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS102  <->  [ok] - 剩余数量：486
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS108  <->  [ok] - 剩余数量：485
https://ecatalog.smpcorp.com/V2/STD/api/part/p

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS187  <->  [ok] - 剩余数量：414
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS190  <->  [ok] - 剩余数量：413
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS174  <->  [ok] - 剩余数量：412
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS197  <->  [ok] - 剩余数量：411
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS195  <->  [ok] - 剩余数量：410
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS198  <->  [ok] - 剩余数量：409
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS194  <->  [ok] - 剩余数量：408
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS199  <->  [ok] - 剩余数量：407
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS201  <->  [ok] - 剩余数量：406
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS303  <->  [ok] - 剩余数量：334
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS305  <->  [ok] - 剩余数量：333
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS296  <->  [ok] - 剩余数量：332
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS306  <->  [ok] - 剩余数量：331
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS293  <->  [ok] - 剩余数量：330
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS308  <->  [ok] - 剩余数量：329
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS298  <->  [ok] - 剩余数量：328
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS311  <->  [ok] - 剩余数量：327
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS307  <->  [ok] - 剩余数量：326
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS399  <->  [ok] - 剩余数量：254
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS396  <->  [ok] - 剩余数量：253
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS400  <->  [ok] - 剩余数量：252
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS403  <->  [ok] - 剩余数量：251
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS404  <->  [ok] - 剩余数量：250
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS406  <->  [ok] - 剩余数量：249
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS408  <->  [ok] - 剩余数量：248
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS410  <->  [ok] - 剩余数量：247
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS412  <->  [ok] - 剩余数量：246
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS515  <->  [ok] - 剩余数量：175
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS512  <->  [ok] - 剩余数量：174
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS517  <->  [ok] - 剩余数量：173
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS518  <->  [ok] - 剩余数量：172
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS519  <->  [ok] - 剩余数量：171
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS520  <->  [ok] - 剩余数量：170
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS522  <->  [ok] - 剩余数量：169
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS513  <->  [ok] - 剩余数量：168
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS523  <->  [ok] - 剩余数量：167
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS625  <->  [ok] - 剩余数量：95
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS628  <->  [ok] - 剩余数量：94
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS631  <->  [ok] - 剩余数量：93
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS632  <->  [ok] - 剩余数量：92
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS633  <->  [ok] - 剩余数量：91
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS629  <->  [ok] - 剩余数量：90
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS630  <->  [ok] - 剩余数量：89
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS634  <->  [ok] - 剩余数量：88
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=APS637  <->  [ok] - 剩余数量：87
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PAR

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=S-1048  <->  [ok] - 剩余数量：16
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=S-2082  <->  [ok] - 剩余数量：15
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=S-2095  <->  [ok] - 剩余数量：14
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=S2398  <->  [ok] - 剩余数量：13
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=S-2038  <->  [ok] - 剩余数量：12
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=S2412  <->  [ok] - 剩余数量：11
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=S2815  <->  [ok] - 剩余数量：10
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=S2817  <->  [ok] - 剩余数量：9
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=S-1911  <->  [ok] - 剩余数量：8
https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=